<a href="https://colab.research.google.com/github/leventdusunceli/Market_Basket_Analysis/blob/main/Market_Basket_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Market Basket Optimisation w/Association Rule Learning Algorithm

In this notebook we're going to perform association rule learning by apriori method to find which items are most frequently bought together in a supermarket.  These type of studies are generally called market basket optimisation.

Dataset used-->[ link](https://www.kaggle.com/datasets/irfanasrullah/groceries/code?resource=download&select=groceries.csv)

Businesses use the findings of market basket optimization studies to create special offers on certain products or to place products that are frequently bought together physically closer to each other. 

For the purpose of our case study we'll build models for identifying item combinations that satisfy the following conditions:

*  Condition 1 - Frequently Bought Together 
 * people who bought left hand side also bought right hands side very very frequently 






* Condition 2 - Niche Product Combinations
 *  product on the left hand side raise the probability of buying another item drastically 


* Condition 3 - Common Items 
 * items that are bought a lot in general



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


Mounted at /content/drive


#Data Preprocessing

In [ ]:
os.chdir("/content/drive/MyDrive/Online_Lecture_Notes/Machine_Learning_AZ/Practice_Folder/Market_Basket_Apriori")
dataset = pd.read_csv("groceries1.csv")
dataset.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The first column of the dataset contains the number of items in the basket for the respective transaction.  We won't need this number while training our model. 

In [ ]:
dataset = dataset.iloc[:,1:]

Define a function for removing NaN values from result dataframes

In [ ]:
#function for removing nan values as the first or second element of the right hand side 

def nan_remover(df):
  for i in range(len(df)):
    if len(df.Right_Hand_Side[i]) < 2:
      if df.Right_Hand_Side[i][0] == "nan":
        df.drop([i],axis=0,inplace=True)
      else:
        continue 
    elif df.Right_Hand_Side[i][0] == "nan":
      df.drop([i],axis=0,inplace=True)
    elif df.Right_Hand_Side[i][1] == "nan":
      df.drop([i],axis=0,inplace=True)  

#Model Building

We'll use the apriori package from apyori to build our Apriori learning algorithm.  


In [ ]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=d9812f5b7f87c0ab00f5032791771c50fed39e3ce1c8fbe6dc995dec7cda6e26
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


*apriori* package takes the input in a list format. Therefore we'll create an empty list and add each row into the list

In [ ]:
dataset.describe()

,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
count,9835,7676,6033,4734,3729,2874,2229,1684,1246,896,...,14,8,7,7,6,5,4,1,1,1
unique,158,151,155,153,150,137,138,140,128,120,...,13,7,6,7,5,4,3,1,1,1
top,sausage,whole milk,whole milk,whole milk,rolls/buns,soda,soda,shopping bags,soda,shopping bags,...,waffles,bottled beer,chocolate,shopping bags,hygiene articles,napkins,house keeping products,skin care,hygiene articles,candles
freq,825,654,506,315,176,150,120,76,61,49,...,2,2,2,1,2,2,2,1,1,1


In [ ]:
transactions = []

for i in range(0,9835): #dataset contains 9835 transactions 
  transactions.append([str(dataset.values[i,j]) for j in range(0,32)])

In the apriori() function we'll define the following: 

* Minimum Support 
* Minimum Confidence 
* Minimum Lift 
* Minimum and maximum number of items we want to pair up

One can change each of these variables to find the most optimal values. We'll also be training our model with different support and confidence levels.


To read more about association rule learning and the terms above, click [here](https://en.wikipedia.org/wiki/Association_rule_learning/) 

##Important note about the dataset

As a prior knowledge we know that the following items are too common in each transaction, therefore their presence in results can misrepresent the importance.  We'll be filtering out results containing these items in certain cases.
* Whole milk 
* Root vegetables
* Other vegetables 
* Tropical fruit
* Yogurt

#Condition 1 - Frequently Bought Together

Our first condition will look for the most strong associations, items that are bought together very very frequently. 


In this condition combinations doesn't have to make up a large portion of all the transactions 






We can inspect for this condition by setting a relatively high mimnum confidence, high minimum lift and low minimum support.

*  **Min Support = .001**
 * Item combination appears in at least %0.1 of the transactions 


* **Min Confidence = 0.7**  
 * Transactions containing left hand side should also contain right hand side at least 60% of the time 

* **Min Lift = 2**
 * Right hand being bought increases the chance of left hand side being bought by at least 2 times

* **Min Length & Max Length** 
 * We'll build 3 different models for finding item combinations containing a maximum of 4,3 and 2 items respectively 
 

###Condition 1, Max_Length=4

In [ ]:
from apyori import apriori 
rules = apriori(transactions = transactions,min_support= 0.001,min_confidence=0.70, min_lift=2,
                min_length=2,max_length=4)

results1 = list(rules)

#code snippet for displaying the rules returned better 
def inspect(results):
    lhs         = [tuple(result[2][0][0]) for result in results]
    rhs         = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

    
resultsinDataFrame = pd.DataFrame(inspect(results1), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

#remove NaN values 
nan_remover(resultsinDataFrame)

#List the top 10 results with decreasing lift
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
11,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
355,"(other vegetables, butter, sugar)","(whipped/sour cream,)",0.001017,0.714286,9.964539
482,"(fruit/vegetable juice, citrus fruit, grapes)","(tropical fruit,)",0.001118,0.846154,8.063879
738,"(other vegetables, fruit/vegetable juice, grapes)","(tropical fruit,)",0.001118,0.785714,7.487888
157,"(citrus fruit, beef, whipped/sour cream)","(root vegetables,)",0.001017,0.769231,7.057262
424,"(citrus fruit, chicken, sausage)","(root vegetables,)",0.001017,0.769231,7.057262
484,"(fruit/vegetable juice, citrus fruit, oil)","(root vegetables,)",0.001017,0.769231,7.057262
951,"(other vegetables, tropical fruit, turkey)","(root vegetables,)",0.001220,0.750000,6.880830
695,"(other vegetables, pip fruit, frozen fish)","(tropical fruit,)",0.001017,0.714286,6.807171
740,"(ham, other vegetables, fruit/vegetable juice)","(tropical fruit,)",0.001017,0.714286,6.807171


The first row of the results shows a great combination.  Other rows contain items that are over-represented.  Let's filter out those items. 




In [ ]:
df_a = pd.DataFrame()
for i in range(len(resultsinDataFrame)): 
  if (resultsinDataFrame.Right_Hand_Side.iloc[i][0]) != "root vegetables":
    df_a = df_a.append(resultsinDataFrame.iloc[i])

df_b = pd.DataFrame()
for i in range(len(df_a)):
  if (df_a.Right_Hand_Side.iloc[i][0]) != "whole milk":
    df_b = df_b.append(df_a.iloc[i])

df_c = pd.DataFrame()
for i in range(len(df_b)):
  if (df_b.Right_Hand_Side.iloc[i][0]) != "other vegetables":
    df_c = df_c.append(df_b.iloc[i])

df_d = pd.DataFrame()
for i in range(len(df_c)):
  if (df_c.Right_Hand_Side.iloc[i][0]) != "tropical fruit":
    df_d = df_d.append(df_c.iloc[i])

condition_1_filtered = pd.DataFrame()
for i in range(len(df_d)):
  if (df_d.Right_Hand_Side.iloc[i][0]) != "yogurt":
    condition_1_filtered = condition_1_filtered.append(df_d.iloc[i])

condition_1_filtered.nlargest(n = 10, columns = 'Lift')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
11,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
355,"(other vegetables, butter, sugar)","(whipped/sour cream,)",0.001017,0.714286,9.964539
51,"(misc. beverages, coffee)","(soda,)",0.001017,0.769231,4.411303
186,"(bottled water, bottled beer, sausage)","(soda,)",0.001118,0.733333,4.205442
108,"(newspapers, spread cheese)","(rolls/buns,)",0.001220,0.750000,4.077529


In [ ]:
condition_1_filtered.to_excel("Condition1_4.xlsx",index= False)

Great bit of results.  Each row offers opportunity for increasing sales.  More analysis in the[ results presentation.](https://github.com/leventdusunceli/Market_Basket_Analysis/blob/main/Results_Presentation.pdf)

###Condition 1, Max_Length=2

We didn't look at max_length=3 because previous condition mostly returned 3 item combinations. 

We'll also lower the min_confidence to 0.5 for this query.

In [ ]:
rules = apriori(transactions = transactions,min_support= 0.001,min_confidence=0.50, min_lift=2,
                min_length=2,max_length=2)

results1 = list(rules)
condition1_3 = pd.DataFrame(inspect(results1), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

#remove NaN values 
nan_remover(condition1_3)

#List the top 10 results with decreasing confidence
condition1_3.nlargest(n = 10, columns = 'Lift')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
4,"(honey,)","(whole milk,)",0.001118,0.733333,2.870009
10,"(tidbits,)","(rolls/buns,)",0.001220,0.521739,2.836542
6,"(rice,)","(other vegetables,)",0.003965,0.520000,2.687441
7,"(specialty cheese,)","(other vegetables,)",0.004270,0.500000,2.584078
1,"(cereals,)","(whole milk,)",0.003660,0.642857,2.515917
9,"(rice,)","(whole milk,)",0.004677,0.613333,2.400371
2,"(cocoa drinks,)","(whole milk,)",0.001322,0.590909,2.312611
8,"(pudding powder,)","(whole milk,)",0.001322,0.565217,2.212062
5,"(jam,)","(whole milk,)",0.002949,0.547170,2.141431
0,"(baking powder,)","(whole milk,)",0.009253,0.522989,2.046793


In [ ]:
#filter whole milk out 
df_a = pd.DataFrame()

for i in range(len(condition1_3)): 
  if (condition1_3.Right_Hand_Side.iloc[i][0]) != "whole milk":
    df_a = df_a.append(condition1_3.iloc[i])

df_a.nlargest(n = 10, columns = 'Lift')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
10,"(tidbits,)","(rolls/buns,)",0.001220,0.521739,2.836542
6,"(rice,)","(other vegetables,)",0.003965,0.520000,2.687441
7,"(specialty cheese,)","(other vegetables,)",0.004270,0.500000,2.584078


Again, interesting combinations. 

In [ ]:
df_a.to_excel("Condition1_2.xlsx",index=False)

#Condition 2 - Niche Product Combinations

In this condition we'll identify items that aren't bought too frequently but, if bought, increases the possibility of buying right hand side exceptionally. 

We'll achieve this by setting a high lift and confidence.




*  **Min Support = 0.0008-.001**
 * Item combination appears in at least 0.0008-.001% of all transactions 


* **Min Confidence = 0.4-0.7**  
 * Transactions containing left hand side should also contain right hand side at least 40 to 70% of the time 

* **Min Lift = 5-7**
 * Right hand being bought increases the chance of left hand side being bought by at least 5-7 times

* **Min Length & Max Length** 
 * We'll build 3 different models for finding item combinations containing a maximum of 4,3 and 2 items respectively 

###Condition 2, Max Length = 4

In [ ]:
rules3 = apriori(transactions = transactions,min_support= 0.001,min_confidence=0.6, min_lift=7,
                min_length=2,max_length=4)

results3 = list(rules3)
    
df3 = pd.DataFrame(inspect(results3), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
nan_remover(df3)

df3.nlargest(n = 10, columns = "Lift")

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(soda, Instant food products)","(hamburger meat,)",0.001220,0.631579,18.995654
4,"(soda, popcorn)","(salty snack,)",0.001220,0.631579,16.697793
3,"(ham, processed cheese)","(white bread,)",0.001932,0.633333,15.045491
27,"(yogurt, rice)","(other vegetables, root vegetables)",0.001423,0.608696,12.846613
30,"(yogurt, rice)","(whole milk, root vegetables)",0.001423,0.608696,12.445991
8,"(yogurt, hamburger meat, whipped/sour cream)","(butter,)",0.001017,0.625000,11.278670
1,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
12,"(other vegetables, butter, sugar)","(whipped/sour cream,)",0.001017,0.714286,9.964539
10,"(butter, whole milk, hard cheese)","(whipped/sour cream,)",0.001423,0.666667,9.300236
23,"(herbs, fruit/vegetable juice)","(other vegetables, whole milk)",0.001017,0.666667,8.908514


Let's filter out most commonly found items

In [ ]:
df_a = pd.DataFrame()
for i in range(len(df3)): 
  if (df3.Right_Hand_Side.iloc[i][0]) != "root vegetables":
    df_a = df_a.append(df3.iloc[i])

df_b = pd.DataFrame()
for i in range(len(df_a)):
  if (df_a.Right_Hand_Side.iloc[i][0]) != "whole milk":
    df_b = df_b.append(df_a.iloc[i])

df_c = pd.DataFrame()
for i in range(len(df_b)):
  if (df_b.Right_Hand_Side.iloc[i][0]) != "other vegetables":
    df_c = df_c.append(df_b.iloc[i])

df_d = pd.DataFrame()
for i in range(len(df_c)):
  if (df_c.Right_Hand_Side.iloc[i][0]) != "tropical fruit":
    df_d = df_d.append(df_c.iloc[i])

df_e = pd.DataFrame()
for i in range(len(df_d)):
  if (df_d.Right_Hand_Side.iloc[i][0]) != "yogurt":
    df_e = df_e.append(df_d.iloc[i])

df_f = pd.DataFrame()
for i in range(len(df_e)):
  if (df_e.Right_Hand_Side.iloc[i][0]) != "whipped/sour cream":
    df_f = df_f.append(df_e.iloc[i])

df_g = pd.DataFrame()
for i in range(len(df_f)):
  if (df_f.Right_Hand_Side.iloc[i][0]) != "pip fruit":
    df_g = df_g.append(df_f.iloc[i])

df_g.nlargest(n = 10, columns = 'Lift')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(soda, Instant food products)","(hamburger meat,)",0.001220,0.631579,18.995654
4,"(soda, popcorn)","(salty snack,)",0.001220,0.631579,16.697793
3,"(ham, processed cheese)","(white bread,)",0.001932,0.633333,15.045491
8,"(yogurt, hamburger meat, whipped/sour cream)","(butter,)",0.001017,0.625000,11.278670
1,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
2,"(specialty chocolate, frozen vegetables)","(fruit/vegetable juice,)",0.001017,0.625000,8.645394
17,"(other vegetables, herbs, tropical fruit)","(citrus fruit,)",0.001017,0.625000,7.551443


Great versatile findings.   

In [ ]:
df_g.to_excel("Condition2_4.xlsx",index=False)

###Condition 2, Max Length= 3, Min Confidence= 0.5

As we look for 3-item combinations, inspecting lower confidence combinations can also provide interesting results

In [ ]:
rules3_2 = apriori(transactions = transactions,min_support= 0.001,min_confidence=0.5, min_lift=7,
                min_length=2,max_length=3)

results3_2 = list(rules3_2)
    
df3_2 = pd.DataFrame(inspect(results3_2), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
nan_remover(df3_2)

df3_2.nlargest(n = 10, columns = "Lift")

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(soda, Instant food products)","(hamburger meat,)",0.001220,0.631579,18.995654
10,"(soda, popcorn)","(salty snack,)",0.001220,0.631579,16.697793
2,"(baking powder, flour)","(sugar,)",0.001017,0.555556,16.408075
8,"(ham, processed cheese)","(white bread,)",0.001932,0.633333,15.045491
1,"(whole milk, Instant food products)","(hamburger meat,)",0.001525,0.500000,15.038226
6,"(processed cheese, domestic eggs)","(white bread,)",0.001118,0.523810,12.443639
3,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
7,"(specialty chocolate, frozen vegetables)","(fruit/vegetable juice,)",0.001017,0.625000,8.645394
9,"(other vegetables, instant coffee)","(whipped/sour cream,)",0.001017,0.526316,7.342292
5,"(butter, hard cheese)","(whipped/sour cream,)",0.002034,0.512821,7.154028


These results can provide insight into the sales approach of hamburger meat.  Detailed insights and thoughts in [results presentation](https://github.com/leventdusunceli/Market_Basket_Analysis/blob/main/Results_Presentation.pdf)

In [ ]:
df3_2.to_excel("Condition2_3.xlsx",index= False)

###Condition 2, Max Length= 2, Min Confidence= 0.4

Checking for item pairs that have a high lift

In [ ]:
rules3_2 = apriori(transactions = transactions,min_support= 0.001,min_confidence=0.4, min_lift=1.1,
                min_length=2,max_length=2)

results3_2 = list(rules3_2)
    
df3_2 = pd.DataFrame(inspect(results3_2), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
nan_remover(df3_2)

df_a = pd.DataFrame()
for i in range(len(df3_2)): 
  if (df3_2.Right_Hand_Side.iloc[i][0]) != "root vegetables":
    df_a = df_a.append(df3_2.iloc[i])

df_b = pd.DataFrame()
for i in range(len(df_a)):
  if (df_a.Right_Hand_Side.iloc[i][0]) != "other vegetables":
    df_b = df_b.append(df_a.iloc[i])

df_c = pd.DataFrame()
for i in range(len(df_b)):
  if (df_b.Right_Hand_Side.iloc[i][0]) != "whole milk":
    df_c = df_c.append(df_b.iloc[i])
df_c.nlargest(n=10, columns = "Lift")

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
5,"(liquor,)","(bottled beer,)",0.004677,0.422018,5.240594
41,"(meat spreads,)","(yogurt,)",0.001830,0.428571,3.072157
62,"(tidbits,)","(rolls/buns,)",0.001220,0.521739,2.836542
68,"(tidbits,)","(soda,)",0.001017,0.434783,2.493345
61,"(skin care,)","(rolls/buns,)",0.001423,0.400000,2.174682


There are still some insights in the results but unfortunately there isn't any item pair that can be classified as having exceptionally high lift.

In [ ]:
df_c.to_excel("Condition2_2.xlsx", index=False)

###Condition 2 - Nichest of Niches

We're going to lower the support even more to find very strongly associated but very rarely bought product combinations

In [ ]:
rules3_1 = apriori(transactions = transactions,min_support= 0.0008,min_confidence=0.75, min_lift=5,
                min_length=2,max_length=4)

results3 = list(rules3_1)
    
df3_lowsupport = pd.DataFrame(inspect(results3), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
nan_remover(df3_lowsupport)

In [ ]:
df_a = pd.DataFrame()
for i in range(len(df3_lowsupport)): 
  if (df3_lowsupport.Right_Hand_Side.iloc[i][0]) != "whole milk":
    df_a = df_a.append(df3_lowsupport.iloc[i])

df_b = pd.DataFrame()
for i in range(len(df_a)):
  if (df_a.Right_Hand_Side.iloc[i][0]) != "other vegetables":
    df_b = df_b.append(df_a.iloc[i])

df_c = pd.DataFrame()
for i in range(len(df_b)):
  if (df_b.Right_Hand_Side.iloc[i][0]) != "root vegetables":
    df_c = df_c.append(df_b.iloc[i])

df_d = pd.DataFrame()
for i in range(len(df_c)):
  if (df_c.Right_Hand_Side.iloc[i][0]) != "yogurt":
    df_d = df_d.append(df_c.iloc[i])

df_e = pd.DataFrame()
for i in range(len(df_d)):
  if (df_d.Right_Hand_Side.iloc[i][0]) != "tropical fruit":
    df_e = df_e.append(df_d.iloc[i])

df_f = pd.DataFrame()
for i in range(len(df_e)):
  if (df_e.Right_Hand_Side.iloc[i][0]) != "whipped/sour cream":
    df_f = df_f.append(df_e.iloc[i])

df_f.nlargest(n=10,columns= "Support")

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
2,"(liquor, red/blush wine)","(bottled beer,)",0.001932,0.904762,11.235269
34,"(tropical fruit, hard cheese, whipped/sour cream)","(butter,)",0.000915,0.818182,14.764804
48,"(margarine, candy, rolls/buns)","(soda,)",0.000915,0.900000,5.161224
17,"(soda, liquor, red/blush wine)","(bottled beer,)",0.000813,1.000000,12.417929
36,"(whole milk, onions, long life bakery product)","(butter,)",0.000813,0.800000,14.436697


In [ ]:
df_f.to_excel("Condition2_nichest.xlsx", index=False)

#Condition 3 - Common Items 

Now let's search for any interesting combinations among the items that are bought a lot.  

We'll perform the search by setting high support and relatively lower confidence.  High lift will be the indicator of a good combination

##Condition 3, Min Support = 0.03, Min Confidence= 0.4, Max Length= 2

In [ ]:
rules3_1 = apriori(transactions = transactions,min_support= 0.03,min_confidence=0.4, min_lift=1.1,
                min_length=2,max_length=2)

results3_1 = list(rules3_1)
    
df3_1 = pd.DataFrame(inspect(results3_1), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
nan_remover(df3_1)

df3_1.nlargest(n = 10, columns = "Lift")

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(root vegetables,)","(other vegetables,)",0.047382,0.434701,2.246605
3,"(whipped/sour cream,)","(whole milk,)",0.032232,0.449645,1.759754
1,"(root vegetables,)","(whole milk,)",0.048907,0.448694,1.756031
2,"(tropical fruit,)","(whole milk,)",0.042298,0.403101,1.577595
4,"(yogurt,)","(whole milk,)",0.056024,0.401603,1.571735


These 5 combinations are going to appear in almost 5% of all baskets. 

In [ ]:
df3_1.to_excel("Condition3.xlsx", index=False)

#Results & Discussion 

For more detailed explanation of rationale behind each condition and for deductions from each condition, refer to the results presentation